# Quickstart Guide
### Use this with README file

In [1]:
import boto3
import xmltodict
import ast

# these are base links to view/access HITs that have been published
MTURK_SANDBOX = "https://mturk-requester-sandbox.us-east-1.amazonaws.com"
MTURK_PROD = "https://mturk-requester.us-east-1.amazonaws.com"
MTURK_PROD_VIZ = "https://worker.mturk.com/mturk/preview?groupId="
MTURK_SB_VIZ = "https://workersandbox.mturk.com/mturk/preview?groupId="

# a toggle to specify whether you're in sandbox mode or not
sandbox = True

# for security purposes, these keys shouldn't be public
ACCESS_KEY_ID = ""
SECRET_ACCESS_KEY = ""

mturk = boto3.client(
    "mturk",
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    region_name="us-east-1",
    endpoint_url=MTURK_SANDBOX
    if sandbox
    else MTURK_PROD,  # if you're in sandbox mode, the client will use the sandbox endpoint
)

# to sanity check that you're in sandbox mode, your account balance will always be $10000
print(
    "I have $"
    + mturk.get_account_balance()["AvailableBalance"]
    + " in my Sandbox account"
)


I have $10000.00 in my Sandbox account


In [ ]:
# this will read in your question interface script
question = open(file="questions.xml", mode="r").read()

# this defines a list of qualification requirements
qr = [
    {
        "QualificationTypeId": "000000000000000000L0",  # Worker Percentage Approval Rate
        "Comparator": "GreaterThanOrEqualTo",
        "IntegerValues": [95],  # 95% approval rate
    },
    {
        "QualificationTypeId": "00000000000000000040",  # Number of HITs Approved
        "Comparator": "GreaterThanOrEqualTo",
        "IntegerValues": [1000],  # At least 1000 HITs completed
    },
    {
        "QualificationTypeId": "00000000000000000071",  # Worker Locale
        "Comparator": "EqualTo",
        "LocaleValues": [{"Country": "US"}],  # Located in the United States
    },
]

In [2]:
# getting the data to show on the question interface
with open("data/imgs.csv", "r") as f:
    data = [line.strip().split(",") for line in f]

# looping through each series of data
# this example uses 10 images for a question
for series in data:
    # replacing the placeholder URL endpoints with the specific data URL endpoints
    for i, img in enumerate(series):
        question = question.replace("img" + str(i + 1) + ".png", img + ".png")

    # creating a new hit after modifying the question template with data
    # you can change the parameters of this function depending on HIT setting
    new_hit = mturk.create_hit(
        Title="Rank a series of images",
        Description="Rank a series images from 1 to 10",
        Keywords="rank, image, quick, labeling, easy, simple, fast",
        Reward="0.45",
        MaxAssignments=1,
        LifetimeInSeconds=1000,
        AssignmentDurationInSeconds=200,
        AutoApprovalDelayInSeconds=2345,
        Question=question,
        QualificationRequirements=[]
        if sandbox
        else qr,  # for sandbox, best not to have requirements so you can access it yourself (if you don't meet the requirements)
    )

    # logging hits published to later access it
    with open("logs/published_hits.txt", "a") as f:
        f.write(new_hit["HIT"]["HITId"] + "\n")


# verbose setting in sandbox that will return a link to see how workers interact with your task
if sandbox:
    print("A new HIT has been created. You can preview it here:")
    print((MTURK_SB_VIZ if sandbox else MTURK_PROD_VIZ) + new_hit["HIT"]["HITGroupId"])
    print("HITID = " + new_hit["HIT"]["HITId"] + " (Use to Get Results)")

A new HIT has been created. You can preview it here:
https://workersandbox.mturk.com/mturk/preview?groupId=33UIGJTG47TXFOULN3JJ0Y4BWIFZ1D
HITID = 3OKP4QVBQKKSDVE4BPDDFAVN5JLAGM (Use to Get Results)


In [3]:
# accessing all the HITs published
with open("logs/published_hits.txt", "r") as f:
    hit_ids = [line.strip() for line in f]

# for this example, we will use the first HIT
hit_id = hit_ids[0]
# if you want to check info about the specific HIT
hit_info = mturk.get_hit(HITId=hit_id)
# looking at results from assignments that have been submitted from a specific HIT
worker_results = mturk.list_assignments_for_hit(
    HITId=hit_id, AssignmentStatuses=["Submitted"]
)["Assignments"]

In [15]:
# iterating through results to get image rankings
for result in worker_results:
    d = xmltodict.parse(result["Answer"])
    image_rankings = ast.literal_eval(
        ast.literal_eval(d["QuestionFormAnswers"]["Answer"]["FreeText"])[0][
            "clickedImages"
        ]
    )

# this should correspond to the rankings given on the image in the README.md file
image_rankings

['https://mturk-guide.s3.us-west-2.amazonaws.com/avocado_chair.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/burrito_human.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/card_mansion.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/carnivore_feast.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/cat_paris.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/eggs_soysauce.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/grape_cake.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/math_dancing.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/strawberry_cake.png',
 'https://mturk-guide.s3.us-west-2.amazonaws.com/yogurt.png']

### How the code chunk works above to unpack the result

In [4]:
d

{'QuestionFormAnswers': {'@xmlns': 'http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd',
  'Answer': {'QuestionIdentifier': 'taskAnswers',
   'FreeText': '[{"clickedImages":"[\\"https://mturk-guide.s3.us-west-2.amazonaws.com/avocado_chair.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/burrito_human.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/card_mansion.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/carnivore_feast.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/cat_paris.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/eggs_soysauce.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/grape_cake.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/math_dancing.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/strawberry_cake.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/yogurt.png\\"]"}]'}}}

In [7]:
d.keys()

dict_keys(['QuestionFormAnswers'])

In [9]:
d["QuestionFormAnswers"].keys()

dict_keys(['@xmlns', 'Answer'])

In [10]:
d["QuestionFormAnswers"]["Answer"]

{'QuestionIdentifier': 'taskAnswers',
 'FreeText': '[{"clickedImages":"[\\"https://mturk-guide.s3.us-west-2.amazonaws.com/avocado_chair.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/burrito_human.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/card_mansion.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/carnivore_feast.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/cat_paris.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/eggs_soysauce.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/grape_cake.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/math_dancing.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/strawberry_cake.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/yogurt.png\\"]"}]'}

In [11]:
d["QuestionFormAnswers"]["Answer"]["FreeText"]

'[{"clickedImages":"[\\"https://mturk-guide.s3.us-west-2.amazonaws.com/avocado_chair.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/burrito_human.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/card_mansion.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/carnivore_feast.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/cat_paris.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/eggs_soysauce.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/grape_cake.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/math_dancing.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/strawberry_cake.png\\",\\"https://mturk-guide.s3.us-west-2.amazonaws.com/yogurt.png\\"]"}]'

In [13]:
import ast
ast.literal_eval(d["QuestionFormAnswers"]["Answer"]["FreeText"])

[{'clickedImages': '["https://mturk-guide.s3.us-west-2.amazonaws.com/avocado_chair.png","https://mturk-guide.s3.us-west-2.amazonaws.com/burrito_human.png","https://mturk-guide.s3.us-west-2.amazonaws.com/card_mansion.png","https://mturk-guide.s3.us-west-2.amazonaws.com/carnivore_feast.png","https://mturk-guide.s3.us-west-2.amazonaws.com/cat_paris.png","https://mturk-guide.s3.us-west-2.amazonaws.com/eggs_soysauce.png","https://mturk-guide.s3.us-west-2.amazonaws.com/grape_cake.png","https://mturk-guide.s3.us-west-2.amazonaws.com/math_dancing.png","https://mturk-guide.s3.us-west-2.amazonaws.com/strawberry_cake.png","https://mturk-guide.s3.us-west-2.amazonaws.com/yogurt.png"]'}]